In [1]:
import pandas as pd
import json
import requests
import aiohttp
import asyncio

## Getting surfline spots

Taken from [ovalwood](https://github.com/ovalwood/surf_locations) and adapted to run a bit faster with async


In [2]:
response = requests.get("https://services.surfline.com/taxonomy?type=taxonomy&id=58f7ed51dadb30820bb3879c&maxDepth=0")
json_data = response.json()
json_contains = json_data['contains']
states = []
state_ids = []
state_urls = []
for x in json_contains:
    states.append(x['name'])
    state_ids.append(x['_id'])


In [3]:

for state_id in state_ids:
    state_urls.append("https://services.surfline.com/taxonomy?type=taxonomy&id=" + state_id + "&maxDepth=0")
   

In [4]:
async def fetch(url, session):
    async with session.get(url) as response:
        return await response.json()

async def fetch_all(urls):
    data = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            tasks.append(fetch(url, session))
        data = await asyncio.gather(*tasks)
    return data

In [5]:
state_data = await fetch_all(state_urls)

In [6]:
county_ids = []    
for state in state_data:
    state_contains = state['contains']
    for y in state_contains:
        county_ids.append(y['_id'])
      

In [7]:
county_urls = []
for county_id in county_ids:
    county_urls.append("https://services.surfline.com/taxonomy?type=taxonomy&id=" + county_id + "&maxDepth=0")


In [8]:

county_data = await fetch_all(county_urls)

In [9]:
region_ids = []
region_names = []
for county in county_data:
    county_contains = county['contains']
    for z in county_contains:
        region_ids.append(z['_id'])
        region_names.append(z['name'])
    

In [10]:
region_urls = []
for region_id in region_ids:
    region_urls.append("https://services.surfline.com/taxonomy?type=taxonomy&id=" + region_id + "&maxDepth=0")


In [11]:
region_data = await fetch_all(region_urls)

In [12]:
def find_val(data, target_key, target_value):
    for key, value in data.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    return item

In [13]:
region_data[0]["geonames"]["adminName1"]


KeyError: 'geonames'

In [15]:
spot_ids = []
spot_names = []
spot_address = []


spot_lon = []
spot_lat = []
spot_urls = []


In [16]:
len(spot_address), len(spot_ids), len(spot_names)

(0, 0, 0)

In [17]:

for region in region_data:
    region_contains = region['contains']
    if len(region_contains) == 0:
        spot_ids.append(region.get("spot", ""))
        spot_names.append(region.get("name", ""))
        spot_address.append("")
        region_associated = region['associated']
        region_links = region_associated['links']
        region_location = region['location']
        region_coordinates = region_location['coordinates']
        spot_lon.append(region_coordinates[0])
        spot_lat.append(region_coordinates[1])
        for i in region_links:
            if i['key'] == "www":
                spot_urls.append(i['href'])

In [18]:
region_data[0].get("contains")[1]

IndexError: list index out of range

In [19]:
df = pd.DataFrame({"ids": spot_ids, "names": spot_names, "lon": spot_lon, "lat": spot_lat, "urls": spot_urls})
df.to_csv('./data/spot_list.csv')

In [20]:
df.head()

,ids,names,lon,lat,urls
0,5842041f4e65fad6a7708aa3,Ft. Pierce Inlet,-80.289140,27.475313,https://www.surfline.com/surf-report/ft-pierce...
1,5842041f4e65fad6a7708aab,Lori Wilson Park,-80.605680,28.336757,https://www.surfline.com/surf-report/lori-wils...
2,5842041f4e65fad6a7708ab1,Vero Pier,-80.360270,27.670122,https://www.surfline.com/surf-report/vero-pier...
3,584204214e65fad6a7709cc4,Millennium Beach Park,-80.580575,28.142895,https://www.surfline.com/surf-report/millenniu...
4,5842041f4e65fad6a7708aa7,Playalinda,-80.623290,28.643973,https://www.surfline.com/surf-report/playalind...


In [ ]:
spot_dict = [{"lat": lat, "lng": lng, "spot_name": name, "street_address": name} for lat, lng, name, name in zip(spot_lat, spot_lon, spot_names, spot_names)]

In [ ]:
legacy_spot_dict = [{"latitude": lat, "longitude": lng, "spot_name": name, "street_address": name} for lat, lng, name, name in zip(spot_lat, spot_lon, spot_names, spot_names)]

In [ ]:
legacy_spot_dict

In [ ]:
spot_dict

In [ ]:
async def post_spot(spot, session, url):
        async with session.post(url, json=spot) as response:
             return await response.text()

async def post_all(spot_dict, url):
    data = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for spot in spot_dict:
            tasks.append(post_spot(spot, session, url))
        data = await asyncio.gather(*tasks)
    return data


In [ ]:
df = pd.DataFrame(legacy_spot_dict)

In [ ]:
df_lat_lon = df[['latitude', 'longitude']]

In [ ]:
df_lat_lon.head(), df_lat_lon.shape

In [ ]:
import numpy as np

In [ ]:
dfs = np.array_split(df, 3)
df_1, df_2, df_3 = dfs[0], dfs[1], dfs[2]


In [ ]:
df_1.head()

In [ ]:
df_1.to_csv('./data/df_1.csv')

In [ ]:
df_2.to_csv('./data/df_2.csv')
df_3.to_csv('./data/df_3.csv')

In [ ]:
# res = await post_all(spot_dict)
# res[0]

In [ ]:
legacy_spot_dict

In [ ]:
processed_files = ["./data/df-1-processed.csv", "./data/df-2-processed.csv", "./data/df-3-processed.csv"]

In [ ]:
dfs = [pd.read_csv(file) for file in processed_files]


In [ ]:
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df['city_state'] = df['city'] + ", " + df["state"]

In [ ]:
df['city_state']

In [ ]:
legacy_spot_dict[:2]

In [ ]:
for idx, spot in enumerate(legacy_spot_dict):
    spot["street_address"] = f"{df['city_state'][idx]}"

In [ ]:
for idx, spot in enumerate(spot_dict):
    spot["street_address"] = f"{df['city_state'][idx]}"

In [ ]:
spot_dict

In [ ]:
legacy_spot_dict[:5]

In [ ]:
with open('./data/legacy_spot_dict.json', 'w') as f:
    json.dump(legacy_spot_dict, f)

In [ ]:
import json

In [ ]:
with open('./data/legacy_spot_dict.json', 'r') as f:
    spot_dict = json.load(f)

In [ ]:
updated_spots = [
    {'lat' if key == 'latitude' else 'lng' if key == 'longitude' else key: value for key, value in spot.items()}
    for spot in spot_dict
]

In [ ]:
updated_spots

In [ ]:
url = 'https://api.peterbull.org/addspot'

In [ ]:
# response = await post_all(updated_spots, url)

In [ ]:
spots_json = json.load('./data/legacy_spot_dict.json')

In [ ]:
len(legacy_spot_dict)